In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

# Chromedriver 경로 설정 (사용자 환경에 맞게 수정)
chromedriver_path = #"yourpath"

def get_etf_data_from_naver(chromedriver_path):
    # Selenium WebDriver 설정
    service = Service(chromedriver_path)
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    # Naver ETF 페이지 접속
    url = "https://finance.naver.com/sise/etf.naver"
    driver.get(url)

    try:
        # 테이블 요소 가져오기
        table = driver.find_element(By.ID, "etfItemTable")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 테이블 헤더 추출
        headers = [header.text.strip() for header in rows[0].find_elements(By.TAG_NAME, "th")]

        # 테이블 데이터 추출
        data = []
        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            # 빈 데이터 제거
            if len(cols) > 0:
                data.append([col.text.strip() for col in cols])

        # DataFrame으로 변환
        df_etf = pd.DataFrame(data, columns=headers)
    except Exception as e:
        driver.quit()
        raise Exception(f"Error while parsing ETF data: {e}")

    driver.quit()
    return df_etf

# 데이터 가져오기
try:
    df_etf = get_etf_data_from_naver(chromedriver_path)
    print("ETF 데이터 상위 5개:")
    print(df_etf.head())  # 상위 5개 출력
except Exception as e:
    print(f"An error occurred: {e}")

ETF 데이터 상위 5개:
                   종목명        현재가   전일비     등락률        NAV  3개월수익률        거래량  \
0                            None  None    None       None    None       None   
1    KODEX CD금리액티브(합성)  1,061,365   285  +0.03%  1,061,263  +0.87%    431,234   
2       TIGER 미국S&P500     21,765    30  +0.14%     21,795  +9.27%  4,788,337   
3  TIGER CD금리투자KIS(합성)     55,625    20  +0.04%     55,622  +0.86%    237,147   
4            KODEX 200     33,775   200  +0.60%     33,836  -1.63%  5,115,740   

  거래대금(백만) 시가총액(억)  
0     None    None  
1  457,695  90,458  
2  104,344  78,017  
3   13,190  62,642  
4  172,675  57,434  


In [36]:
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

# Chromedriver 경로 설정 (사용자 환경에 맞게 수정)
chromedriver_path = #"yourpath"

# 크롤링 함수
def get_etf_data_from_naver(chromedriver_path):
    # Selenium WebDriver 설정
    service = Service(chromedriver_path)
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    # Naver ETF 페이지 접속
    url = "https://finance.naver.com/sise/etf.naver"
    driver.get(url)

    try:
        # 테이블 요소 가져오기
        table = driver.find_element(By.ID, "etfItemTable")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 테이블 헤더 추출
        headers = [header.text.strip() for header in rows[0].find_elements(By.TAG_NAME, "th")]

        # 테이블 데이터 추출
        data = []
        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) > 0:
                data.append([col.text.strip() for col in cols])

        # DataFrame으로 변환
        df_etf = pd.DataFrame(data, columns=headers)

        # 데이터 정리: NaN 대체 및 숫자형 변환
        df_etf["현재가"] = df_etf["현재가"].str.replace(",", "").fillna("0").astype(float)
        df_etf["전일비"] = df_etf["전일비"].str.replace(",", "").fillna("0").astype(float)
        df_etf["거래량"] = df_etf["거래량"].str.replace(",", "").fillna("0").astype(float)

        # 필요한 컬럼만 선택
        df_etf = df_etf.rename(columns={
            "종목명": "etf_name", 
            "현재가": "current_price", 
            "전일비": "price_change", 
            "거래량": "trade_volume"
        })
        df_etf = df_etf[["etf_name", "current_price", "price_change", "trade_volume"]]

    except Exception as e:
        driver.quit()
        raise Exception(f"Error while parsing ETF data: {e}")

    driver.quit()
    return df_etf

# MySQL 데이터베이스 및 테이블 설정
def setup_database_and_table():
    db = pymysql.connect(
        host="localhost",
        port=3306,
        user="name", #default=root
        passwd="password",
        charset="utf8"
    )
    cursor = db.cursor()

    # 데이터베이스 생성
    cursor.execute("CREATE DATABASE IF NOT EXISTS ETF;")
    cursor.execute("USE ETF;")

    # 테이블 생성
    SQL_CREATE_TABLE = """
    CREATE TABLE IF NOT EXISTS naver_etf_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        etf_name VARCHAR(255),
        current_price FLOAT,
        price_change FLOAT,
        trade_volume BIGINT
    );
    """
    cursor.execute(SQL_CREATE_TABLE)

    db.commit()
    return db, cursor

# 데이터 삽입
def insert_etf_data_to_mysql(df, cursor):
    SQL_INSERT = """
    INSERT INTO naver_etf_data (etf_name, current_price, price_change, trade_volume)
    VALUES (%s, %s, %s, %s);
    """

    for _, row in df.iterrows():
        # NaN 값 확인 및 스킵
        if row.isnull().any():
            print(f"NaN 데이터 스킵: {row}")
            continue

        # 데이터 삽입
        try:
            cursor.execute(SQL_INSERT, tuple(row))
        except Exception as e:
            print(f"데이터 삽입 오류: {e}")

# 실행
try:
    # 크롤링
    df_etf = get_etf_data_from_naver(chromedriver_path)
    print("ETF 데이터 상위 5개:")
    print(df_etf.head())

    # MySQL 저장
    db, cursor = setup_database_and_table()
    insert_etf_data_to_mysql(df_etf, cursor)
    db.commit()
    db.close()

    print("MySQL Saved")

except Exception as e:
    print(f"An error occurred: {e}")

ETF 데이터 상위 5개:
              etf_name  current_price  price_change  trade_volume
0                                 0.0           0.0           0.0
1    KODEX CD금리액티브(합성)      1061365.0         285.0      431234.0
2       TIGER 미국S&P500        21765.0          30.0     4788337.0
3  TIGER CD금리투자KIS(합성)        55625.0          20.0      237147.0
4            KODEX 200        33775.0         200.0     5115740.0
✅ ETF 데이터 MySQL 저장 완료!
